# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298388758004                   -0.85    5.4
  2   -8.300178449940       -2.75       -1.25    1.0
  3   -8.300445467725       -3.57       -1.88    2.4
  4   -8.300462681556       -4.76       -2.75    3.0
  5   -8.300464353622       -5.78       -3.13    4.4
  6   -8.300464497468       -6.84       -3.28    1.6
  7   -8.300464567097       -7.16       -3.41    4.0
  8   -8.300464607600       -7.39       -3.54    1.9
  9   -8.300464633905       -7.58       -3.73    1.0
 10   -8.300464640987       -8.15       -3.92    1.0
 11   -8.300464643752       -8.56       -4.29    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67376730692                   -0.70    6.0
  2   -16.67827346691       -2.35       -1.14    1.1
  3   -16.67922860076       -3.02       -1.87    3.9
  4   -16.67928156706       -4.28       -2.77    3.0
  5   -16.67928610583       -5.34       -3.18    4.8
  6   -16.67928621184       -6.97       -3.50    3.2
  7   -16.67928622190       -8.00       -3.96    1.0
  8   -16.67928622421       -8.64       -4.58    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32547997027                   -0.56    6.8
  2   -33.33274173573       -2.14       -1.00    1.0
  3   -33.33415048763       -2.85       -1.74    4.8
  4   -33.33429566353       -3.84       -2.66    4.0
  5   -33.33570876472       -2.85       -2.36    7.8
  6   -33.33694334449       -2.91       -2.53    5.8
  7   -33.33694351252       -6.77       -2.55    1.1
  8   -33.33689208016   +   -4.29       -2.46    2.0
  9   -33.33686441371   +   -4.56       -2.38    1.0
 10   -33.33686047787   +   -5.40       -2.37    1.0
 11   -33.33686246488       -5.70       -2.38    1.0
 12   -33.33686732745       -5.31       -2.39    1.0
 13   -33.33686701025   +   -6.50       -2.39    1.0
 14   -33.33686754410       -6.27       -2.39    1.0
 15   -33.33686547281   +   -5.68       -2.39    1.0
 16   -33.33686998319       -5.35       -2.40    1.0
 17   -33.33693557973       -4.18       -2.82 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298509048501                   -0.85    4.9
  2   -8.300251558922       -2.76       -1.58    1.0
  3   -8.300264900688       -4.87       -2.09    6.1
  4   -8.300342890829       -4.11       -2.21    2.6
  5   -8.300464292146       -3.92       -3.64    1.0
  6   -8.300464549367       -6.59       -3.81    3.8
  7   -8.300464630340       -7.09       -4.17    1.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32378054750                   -0.56    7.0
  2   -33.30444122888   +   -1.71       -1.26    1.2
  3   -5.766900213066   +    1.44       -0.35    8.5
  4   -33.31209170947        1.44       -1.57    7.1
  5   -33.03160074769   +   -0.55       -1.22    5.8
  6   -28.48622923060   +    0.66       -0.72    5.4
  7   -33.33487361101        0.69       -2.22    4.9
  8   -33.33157613377   +   -2.48       -2.04    3.1
  9   -33.33374153379       -2.66       -2.11    2.8
 10   -33.33553286682       -2.75       -2.46    2.5
 11   -33.33669041379       -2.94       -2.81    1.9
 12   -33.33675092251       -4.22       -2.89    2.5
 13   -33.33693281911       -3.74       -3.26    3.2
 14   -33.33693965456       -5.17       -3.56    3.1
 15   -33.33694313817       -5.46       -3.92    3.1
 16   -33.33694390342       -6.12       -4.40    2.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.